In [ ]:
# 导入必要的库
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import torch
import torch.nn as nn
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import torch.nn.functional as F
import nbimporter
from network import *
import os
from utils import *
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'torch'

In [ ]:
seed=42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# 读取制表符分隔的CSV文件
Methylation = pd.read_csv('../../DATA/BRCA_survival/meth.csv')
# 显示 DataFrame
print(Methylation.shape)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.141733,0.106910,0.153344,0.493644,0.257636,0.364777,0.219981,0.252259,0.440933,0.539385,...,0.582180,0.122747,0.552270,0.000000,0.472881,0.277853,0.450420,0.411714,0.346607,0.226955
1,0.028390,0.000000,0.025907,0.412089,0.339159,0.326409,0.222676,0.185724,0.374486,0.467301,...,0.458928,0.080770,0.555985,0.000000,0.453428,0.246680,0.334347,0.376282,0.275763,0.107634
2,0.174621,0.119444,0.144015,0.472943,0.271975,0.361000,0.229209,0.221335,0.429724,0.526628,...,0.536246,0.159083,0.623637,0.000000,0.497386,0.282840,0.459338,0.405583,0.337160,0.222342
3,0.090002,0.090814,0.120774,0.400915,0.336384,0.306688,0.187426,0.226630,0.313737,0.474120,...,0.463836,0.081633,0.578060,0.000000,0.501140,0.267594,0.528654,0.419395,0.273644,0.161116
4,0.076376,0.142307,0.124047,0.425849,0.137060,0.304390,0.171828,0.135269,0.431048,0.441856,...,0.446534,0.081633,0.564433,0.051172,0.414318,0.232996,0.479574,0.380276,0.285492,0.148478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.094807,0.082492,0.042472,0.443077,0.211947,0.317140,0.168021,0.188690,0.397753,0.453853,...,0.503894,0.108356,0.410110,0.007266,0.416697,0.219640,0.508766,0.398576,0.322018,0.154404
66,0.132236,0.106181,0.060387,0.449694,0.173696,0.311363,0.216193,0.189526,0.367328,0.497913,...,0.474982,0.116748,0.634347,0.018202,0.449184,0.206464,0.509352,0.358338,0.268579,0.142307
67,0.094017,0.109788,0.089186,0.492175,0.180054,0.393041,0.249496,0.314460,0.467042,0.501088,...,0.578979,0.117427,0.656068,0.034420,0.463372,0.264348,0.453144,0.342183,0.305735,0.196056
68,0.052224,0.114694,0.053269,0.441395,0.206836,0.331480,0.254972,0.185724,0.399495,0.506450,...,0.478019,0.106181,0.626859,0.000000,0.445341,0.184434,0.417602,0.375327,0.273169,0.100981


In [ ]:
# 读取制表符分隔的CSV文件
miRNASeq = pd.read_csv('../../DATA/BRCA_survival/miRNA.csv')
# 显示 DataFrame
miRNASeq

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.552139,0.559067,0.285569,0.718629,0.390478,0.833890,0.539042,0.403945,0.461322,0.546504,...,0.613052,0.203944,0.637093,0.555595,0.659333,0.533907,0.526485,0.326518,0.441615,0.345450
1,0.580002,0.529317,0.329242,0.738768,0.388183,0.904719,0.580431,0.392392,0.485232,0.548183,...,0.686109,0.231781,0.617823,0.544666,0.661086,0.507549,0.544031,0.303962,0.447308,0.336100
2,0.561408,0.513636,0.315951,0.673154,0.318711,0.922501,0.553679,0.307780,0.455743,0.537512,...,0.580173,0.162286,0.585571,0.492083,0.634090,0.512798,0.561921,0.270619,0.345588,0.301247
3,0.549919,0.462672,0.340425,0.611309,0.316472,0.861999,0.563730,0.292498,0.400651,0.465073,...,0.590594,0.206259,0.554519,0.459943,0.650416,0.446624,0.499137,0.285911,0.324034,0.295996
4,0.567981,0.555092,0.297521,0.694480,0.385651,0.943919,0.565570,0.351122,0.397546,0.525114,...,0.621068,0.232070,0.579691,0.507443,0.663793,0.517530,0.549106,0.347407,0.421452,0.346779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.580880,0.497651,0.258057,0.701615,0.336419,0.864107,0.596609,0.327971,0.499936,0.541747,...,0.639236,0.218292,0.619704,0.499202,0.675908,0.486819,0.553745,0.334531,0.430577,0.311705
66,0.604719,0.491897,0.274434,0.656380,0.357742,0.839752,0.502004,0.357297,0.431040,0.520637,...,0.620938,0.178357,0.582983,0.498561,0.620178,0.492380,0.497586,0.306908,0.383063,0.311347
67,0.600014,0.562987,0.249852,0.701769,0.402879,0.927926,0.572836,0.419111,0.478940,0.564210,...,0.632291,0.264023,0.658148,0.526569,0.683633,0.515043,0.483666,0.328925,0.417604,0.359054
68,0.580941,0.550428,0.367881,0.691000,0.394067,0.853389,0.531384,0.400435,0.448872,0.559928,...,0.623368,0.207858,0.662403,0.559818,0.682088,0.512996,0.504750,0.343361,0.445321,0.356560


In [ ]:
# 读取制表符分隔的CSV文件
RNAseq = pd.read_csv('../../DATA/BRCA_survival/mRNA.csv')
# 显示 DataFrame
RNAseq

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.822696,0.749512,0.738373,0.829757,0.725952,0.510500,0.928945,0.401692,0.279266,0.673976,...,0.337899,0.256514,0.289538,0.182423,0.265771,0.254107,0.222156,0.348486,0.386689,0.317780
1,0.807597,0.717130,0.739479,0.836040,0.743131,0.509932,0.938481,0.344084,0.228623,0.689352,...,0.273958,0.292204,0.217053,0.243807,0.287193,0.272302,0.274035,0.366102,0.316983,0.308586
2,0.808222,0.711237,0.751831,0.856981,0.768841,0.529645,0.954614,0.372392,0.335992,0.727246,...,0.379027,0.326835,0.305716,0.280504,0.323942,0.273627,0.195085,0.405561,0.359562,0.338459
3,0.807853,0.739236,0.756555,0.838984,0.749880,0.512322,0.946342,0.361297,0.228889,0.679450,...,0.357983,0.246972,0.261173,0.204835,0.185487,0.220200,0.183395,0.291305,0.310701,0.277985
4,0.807047,0.729387,0.763370,0.818535,0.726867,0.520051,0.940714,0.412594,0.297168,0.677224,...,0.402075,0.205109,0.222423,0.211773,0.076387,0.233315,0.283959,0.347730,0.391543,0.266929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.815769,0.725591,0.742734,0.825777,0.741116,0.514395,0.954288,0.362880,0.302591,0.675439,...,0.350479,0.243489,0.283342,0.271770,0.268658,0.197210,0.208597,0.324356,0.399081,0.329099
66,0.817828,0.735251,0.729180,0.812079,0.723800,0.484208,0.912951,0.364798,0.114950,0.676715,...,0.295223,0.346479,0.293446,0.316083,0.347091,0.259914,0.304735,0.333377,0.409943,0.248880
67,0.815332,0.737245,0.754298,0.844369,0.749922,0.552492,0.947020,0.364876,0.255961,0.707385,...,0.329426,0.262290,0.244829,0.186200,0.235827,0.133048,0.184496,0.297047,0.374515,0.217278
68,0.795091,0.698683,0.734965,0.843130,0.741016,0.537043,0.938839,0.355818,0.268145,0.689790,...,0.291312,0.200119,0.177841,0.217369,0.306494,0.246859,0.316634,0.321032,0.332343,0.248622


In [ ]:
# print(Methylation.head())
# print(RNAseq.head())
# print(miRNASeq.head())


In [ ]:
# 读取制表符分隔的CSV文件
labels = pd.read_csv('../../DATA/BRCA_survival/labels.csv', sep='\t',header=None)
# 显示 DataFrame
print(labels.shape)

0  
0.0    36
1.0    34
Name: count, dtype: int64

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

# 假设已准备好三个DataFrame：Methylation, RNAseq, miRNASeq 和 labels（仅一列）

# 合并多组组学数据
data_combined = pd.concat([Methylation, RNAseq, miRNASeq], axis=1)

# t-SNE降维
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, max_iter=1000, random_state=42)
tsne_result = tsne.fit_transform(data_combined)

# 构建 t-SNE 可视化 DataFrame
tsne_df = pd.DataFrame(tsne_result, columns=['TSNE1', 'TSNE2'])

# 标签映射（0 = 存活，1 = 死亡）
label_mapping = {
    0: 'Survival within 3.5 years',
    1: 'Death within 3.5 years of survival'
}
tsne_df['Label'] = labels.values.ravel()
tsne_df['Label'] = tsne_df['Label'].map(label_mapping)

# 自定义配色（红蓝对比明确）
custom_palette = {
    'Survival within 3.5 years': '#1f77b4',         # 蓝色
    'Death within 3.5 years of survival': '#d62728'  # 红色
}

# 设置可视化风格
plt.figure(figsize=(10, 7))
sns.set(style="whitegrid")
scatter = sns.scatterplot(
    x='TSNE1', y='TSNE2',
    hue='Label',
    palette=custom_palette,
    data=tsne_df,
    legend='full',
    alpha=0.7,
    s=70
)

# 删除图例标题
legend = plt.legend(title=None, fontsize=20, loc='best')
legend.set_title(None)

# # 添加聚类中心标注（可选）
# centers = tsne_df.groupby('Label')[['TSNE1', 'TSNE2']].mean()
# for label, (x, y) in centers.iterrows():
#     plt.text(x, y, label, fontsize=11, weight='bold', ha='center', va='center',
#              bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'))

# 标题 & 坐标轴样式
plt.title('Multi-Omics Raw Data', fontsize=20, fontweight='bold')
plt.xlabel('TSNE1', fontsize=20)
plt.ylabel('TSNE2', fontsize=20)

# 网格线优化
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.6)

# 图像保存（PNG + 矢量版）
plt.tight_layout()
plt.savefig('tsne_visualization.png', dpi=300, bbox_inches='tight')
plt.show()


NameError: name 'pd' is not defined

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

# 读取多组学特征与标签
data_combined = pd.read_csv('feature.csv')
labels = pd.read_csv('all_probs.csv')

# t-SNE降维
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, max_iter=1000, random_state=42)
tsne_result = tsne.fit_transform(data_combined)

# 构建结果DataFrame
tsne_df = pd.DataFrame(tsne_result, columns=['TSNE1', 'TSNE2'])

# 标签映射
label_mapping = {
    0: 'Survival within 3.5 years',
    1: 'Death within 3.5 years of survival'
}
tsne_df['Label'] = labels.values.ravel()
tsne_df['Label'] = tsne_df['Label'].map(label_mapping)

# 自定义调色板（红蓝）
custom_palette = {
    'Survival within 3.5 years': '#1f77b4',         # 蓝色
    'Death within 3.5 years of survival': '#d62728'  # 红色
}

# 绘图设置
plt.figure(figsize=(10, 7))
sns.set(style="whitegrid")
sns.scatterplot(
    x='TSNE1', y='TSNE2',
    hue='Label',
    palette=custom_palette,
    data=tsne_df,
    alpha=0.7,
    s=70
)

# 图像标题
plt.title('MOAT-GPNet Processed Multi-Omics Data', fontsize=20, fontweight='bold')

# 删除图例标题（关键优化）
legend = plt.legend(title=None, fontsize=20, loc='best')
legend.set_title(None)

# 坐标轴样式
plt.xlabel('TSNE1', fontsize=20)
plt.ylabel('TSNE2', fontsize=20)

# 网格线增强
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.6)

# 保存高分辨率图像
plt.tight_layout()
plt.savefig('tsne_visualization_MOAT-GPNet.png', dpi=300, bbox_inches='tight')

# 显示图像
plt.show()
